In [ ]:
import pandas as pd
# Importing the required libraries for plots.
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("CarPrice_Assignment.csv")
df.head()

In [ ]:
#spilting the CarName into car_company and car_model
df["car_company"],df["car_model"] = df["CarName"].str.split(" ",1).str

In [ ]:
# Converting the categorical to numeric
df["fueltype"] = df["fueltype"].map({"gas": 0, "diesel": 1})  # GAS == 0 , DIESEL == 1
df["aspiration"] = df["aspiration"].map({"std":0,"turbo":1})   # STD == 0 , TURBO == 1
df["doornumber"] =df["doornumber"].map({"two":2,"four":4})     # TWO == 2 , FOUR == 4
df["drivewheel"] = df["drivewheel"].map({"rwd":0,"fwd":1})     # rwd == 0 , fwd == 1
df["enginelocation"] = df["enginelocation"].map({"front":1,"rear":0})  # front == 1 , rear == 0
df["cylindernumber"] = df["cylindernumber"].map({"four":4,"six":6,"five":5,"three":3,"twelve":12,"two":2,"eight":8})

In [ ]:
# So know we have all unique values in the categorical data
# Drop the carid column
df.drop(columns=["car_ID","CarName","car_model"],inplace=True)
df.head()

In [ ]:
# get dummies
#car_body
car_body_status = pd.get_dummies(df["carbody"],drop_first=True)
df = pd.concat([df,car_body_status],axis=1)
#engine_type
engine_type_status = pd.get_dummies(df["enginetype"],drop_first=True)
df = pd.concat([df,engine_type_status],axis=1)
#fuel_system
fuel_system_status = pd.get_dummies(df["fuelsystem"],drop_first=True)
df = pd.concat([df,fuel_system_status],axis=1)


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df["car_company"])
df[["car_company"]]=le.fit_transform(df["car_company"])

In [ ]:
df.drop(columns=["carbody","fueltype","fuelsystem","enginetype"],inplace=True)

In [ ]:
plt.subplots(figsize=(30,50))
sns.heatmap(df.corr(), annot = True, cmap="YlGnBu")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively

df_train, df_test = train_test_split(df, train_size = 0.8, test_size = 0.2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ["symboling","wheelbase","carlength","carwidth","carheight","curbweight","enginesize","boreratio","stroke","compressionratio","horsepower","peakrpm","citympg","highwaympg","price"]

df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

df_train.head()

In [ ]:
y_train = df_train.pop('price')
X_train = df_train

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output number of the variable equal to 10
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 10)             # running RFE
rfe = rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[col]

In [ ]:
# Adding a constant variable 
import statsmodels.api as sm  
X_train_rfe = sm.add_constant(X_train_rfe)

In [ ]:
lm = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model

In [ ]:
#Let's see the summary of our linear model
print(lm.summary())

In [ ]:
X_train_new = X_train_rfe.drop(["curbweight"], axis = 1)

In [ ]:
# Adding a constant variable 
import statsmodels.api as sm  
X_train_lm = sm.add_constant(X_train_new)

In [ ]:
lm = sm.OLS(y_train,X_train_lm).fit()   # Running the linear model

In [ ]:
#Let's see the summary of our linear model
print(lm.summary())

In [ ]:
X_train_rfe.columns

In [ ]:
X_train_new = X_train_rfe.drop(['const'], axis=1)

In [ ]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_new1 = X_train_new.drop(['curbweight'], axis=1)

In [ ]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new1
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
X_train_new3 = X_train_new1.drop(['enginesize'], axis=1)

In [ ]:
# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new3
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
y_train_price = lm.predict(X_train_lm)

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_price), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)                  # Plot heading 
plt.xlabel('Errors', fontsize = 18)                         # X-label

In [ ]:
num_vars = ["symboling","wheelbase","carlength","carwidth","carheight","curbweight","enginesize","boreratio","stroke","compressionratio","horsepower","peakrpm","citympg","highwaympg","price"]
df_test[num_vars] = scaler.transform(df_test[num_vars])

In [ ]:
y_test = df_test.pop('price')
X_test = df_test

In [ ]:
# Now let's use our model to make predictions.

# Creating X_test_new dataframe by dropping variables from X_test
X_test_new = X_test[X_train_new1.columns]

# Adding a constant variable 
X_test_new = sm.add_constant(X_test_new)

In [ ]:
# Making predictions
y_pred = lm.predict(X_test_new)

In [ ]:
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test,y_pred)
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)                          # Y-label

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)